# Nouvelle section

In [0]:
!tar -xf training_set.tar

In [0]:
from os import listdir
from os.path import isfile, join
mypath="training_set"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [0]:
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('nbagg')

import matplotlib.pyplot as plt
plt.rcParams.update({'figure.max_open_warning': 0})

import seaborn as sns
sns.set_style('whitegrid')
import os
from scipy import sparse
from scipy.sparse import csr_matrix

from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
import random

In [0]:
cd training_set

/content/download/training_set


**Creation du dataframe à partir de nos fichiers**


In [0]:
from datetime import datetime
import os
start = datetime.now()
if not os.path.isfile('data.csv'):
    # Create a file 'data.csv' before reading it
    # Read all the files in netflix and store them in one big file('data.csv')
    # reading from each of the four files and appending each rating to a global file 'train.csv'
    data = open('data.csv', mode='w')
    
    row = list()
    files=onlyfiles
    i=0
    for file in files:

        
        #print(file)
        #print("Reading ratings from {}...".format(file))
        with open(file) as f:
            for line in f: 
                del row[:]
                line = line.strip()
                if line.endswith(':'):
                    # All below are ratings for this movie, until another movie appears.
                    movie_id = line.replace(':', '')
                else:
                    row = [x for x in line.split(',')]
                    row.insert(0, movie_id)
                    data.write(','.join(row))
                    data.write('\n')
        i=i+1
    data.close()
print('Time taken :', datetime.now() - start)

In [0]:
df = pd.read_csv('../data.csv', sep=',', 
                       names=['movie', 'user','rating','date'])
df.date = pd.to_datetime(df.date)


# ordonner le rating selon la data.

df.sort_values(by='date', inplace=True)


In [0]:
df.shape


(100480507, 4)

In [0]:
df.head()

,movie,user,rating,date
55210107,16465,510180,3,1999-11-11
30917400,15105,510180,4,1999-11-11
12199060,15894,510180,3,1999-11-11
8628919,9798,510180,3,1999-11-11
55544329,15057,510180,5,1999-11-11


**Verfication des valeurs null**

In [0]:
print("aucune valeur null dans notre dataframe : ", sum(df.isnull().any()))

aucune valeur null dans notre dataframe :  0


**Suppression des duplicatats dans notre dataframe**

In [0]:
dup_bool = df.duplicated(['movie','user','rating'])
dups = sum(dup_bool) # by considering all columns..( including timestamp)
print("Il y a  {} valeur dupliquer dans nos data..".format(dups))

Il y a  0 valeur dupliquer dans nos data..


**Creation de la matrice D**

In [0]:
start = datetime.now()
if os.path.isfile('train_sparse_matrix.npz'):
    print("Si D existe on la charge du disque")
    # just get it from the disk instead of computing it
    train_sparse_matrix = sparse.load_npz('train_sparse_matrix.npz')
    print("DONE ")
else: 
    print("On créer la matrice D depuis le dataframe")
    # create sparse_matrix and store it for after usage.
    # csr_matrix(data_values, (row_index, col_index), shape_of_matrix)
    # It should be in such a way that, MATRIX[row, col] = data
    sparse_matrix = sparse.csr_matrix((df.rating.values, (df.user.values,
                                               df.movie.values)),)
    
    print('Done. Sa forme : (user, movie) : ',sparse_matrix.shape)
    print('Sauvegarde dans le disque')
    # save it into disk
    sparse.save_npz("sparse_matrix.npz", sparse_matrix)
    print('Done \n')

print(datetime.now() - start)

On créer la matrice D depuis le dataframe
Done. Sa forme : (user, movie) :  (2649430, 17771)
Sauvegarde dans le disque
Done 

0:01:31.407458


**On supprimme les colonnes entiérement nule**

In [0]:
 import scipy
 D = scipy.sparse.csr_matrix(sparse_matrix)

**On supprimme les lignes null**

In [0]:
D = sparse_matrix[sparse_matrix.getnnz(1)>0]

**On save notre matrix**

In [0]:
sparse.save_npz("D.npz", D)

In [0]:
mv D.npz ../

In [0]:
sparse_matrix1 = sparse.load_npz('../D.npz')
print('Done. Sa forme : (user, movie) : ',sparse_matrix1.shape)

Done. Sa forme : (user, movie) :  (480189, 17771)


In [0]:
rm probe.csv

**Construire la matrice T**

In [0]:
from datetime import datetime
import os
start = datetime.now()
if not os.path.isfile('probe.csv'):
    # Create a file 'data.csv' before reading it
    # Read all the files in netflix and store them in one big file('data.csv')
    # reading from each of the four files and appending each rating to a global file 'train.csv'
    data = open('probe.csv', mode='w')
    
    row = list()
    files=["../probe.txt"]
    i=0
    for file in files:
        print("do")
        if(i%1000==0):
           print(i)
        
        #print(file)
        #print("Reading ratings from {}...".format(file))
        with open(file) as f:
            for line in f: 
                del row[:]
                line = line.strip()
                if line.endswith(':'):
                    # All below are ratings for this movie, until another movie appears.
                    movie_id = line.replace(':', '')
                else:
                    row = [x for x in line.split(',')]
                    row.insert(0, movie_id)
                    data.write(','.join(row))
                    data.write('\n')
        i=i+1
    data.close()
print('Time taken :', datetime.now() - start)

do
0
Time taken : 0:00:01.959681


In [0]:
mv probe.csv ../

In [0]:
df_probe = pd.read_csv('../probe.csv', sep=',', 
                       names=['movie', 'user'])



# ordonner le rating selon la data.

df_probe.head()

,movie,user
0,1,30878
1,1,2647871
2,1,1283744
3,1,2488120
4,1,317050


In [0]:
common = df.merge(df_probe,on=['user','movie'])

In [89]:

if os.path.isfile('test_sparse_matrix.npz'):
    print("Si T existe on la charge du disque")
    # just get it from the disk instead of computing it
    test_sparse_matrix = sparse.load_npz('T.npz')
    print("DONE ")
else: 
    print("On créer la matrice T depuis le dataframe")
    # create sparse_matrix and store it for after usage.
    # csr_matrix(data_values, (row_index, col_index), shape_of_matrix)
    # It should be in such a way that, MATRIX[row, col] = data
    test_sparse_matrix = sparse.csr_matrix((common.rating.values, (common.user.values,
                                               common.movie.values)),)
    
    print('Done. Sa forme : (user, movie) : ',test_sparse_matrix.shape)
    print('Sauvegarde dans le disque')
    # save it into disk
    sparse.save_npz("T.npz", test_sparse_matrix)
    print('Done \n')



On créer la matrice T depuis le dataframe
Done. Sa forme : (user, movie) :  (2649430, 17771)
Sauvegarde dans le disque
Done 



In [0]:
col= [ i for i in range(17771) if i!=0 ]


In [79]:
#col=np.where(test_sparse_matrix.sum(axis=0)!=0)[1]
test_sparse_matrix=test_sparse_matrix[:,col]
T = test_sparse_matrix[df.user.unique()]
print('Done. Sa forme : (user, movie) : ',T.shape)


Done. Sa forme : (user, movie) :  (480189, 17770)


In [87]:
test_sparse_matrix1 = sparse.load_npz('T.npz')
print('Done. Sa forme : (user, movie) : ',test_sparse_matrix1.shape)

Done. Sa forme : (user, movie) :  (480189, 17770)


In [0]:
mv T.npz ../

**Matrice R**

In [0]:
df1=df.iloc[:,0:2]

In [0]:
R_df=pd.merge(df1,df_probe, indicator=True, how='outer').query('_merge=="left_only"').drop('_merge', axis=1)

In [96]:
R_df.shape

(99072112, 2)

In [97]:
R_df = df.merge(R_df,on=['user','movie'])
R_df.shape

(99072112, 4)

In [98]:
if os.path.isfile('train_sparse_matrix.npz'):
    print("Si R existe on la charge du disque")
    # just get it from the disk instead of computing it
    train_sparse_matrix = sparse.load_npz('R.npz')
    print("DONE ")
else: 
    print("On créer la matrice R depuis le dataframe")
    # create sparse_matrix and store it for after usage.
    # csr_matrix(data_values, (row_index, col_index), shape_of_matrix)
    # It should be in such a way that, MATRIX[row, col] = data
    train_sparse_matrix = sparse.csr_matrix((R_df.rating.values, (R_df.user.values,
                                               R_df.movie.values)),)
    
    print('Done. Sa forme : (user, movie) : ',train_sparse_matrix.shape)
    print('Sauvegarde dans le disque')
    # save it into disk
    sparse.save_npz("R.npz", train_sparse_matrix)
    print('Done \n')


On créer la matrice R depuis le dataframe
Done. Sa forme : (user, movie) :  (2649430, 17771)
Sauvegarde dans le disque
Done 



In [0]:
mv R.npz ../

In [100]:
 import scipy
 M = scipy.sparse.csr_matrix(train_sparse_matrix)
 M.shape

(2649430, 17771)

In [0]:
col=np.where(train_sparse_matrix.sum(axis=0)!=0)[1]
train_sparse_matrix=train_sparse_matrix[:,col]
R = train_sparse_matrix[train_sparse_matrix.getnnz(1)>0]
sparse.save_npz("../R.npz", R)

In [102]:
train_sparse_matrix1 = sparse.load_npz('../R.npz')
print('Done. Sa forme : (user, movie) : ',train_sparse_matrix1.shape)

Done. Sa forme : (user, movie) :  (480189, 17770)
